In [1]:
import ast
import json
import re

import pandas as pd
import pymorphy2
import torch
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

from tqdm.notebook import tqdm

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
def pymorphy2_311_hotfix():
    from inspect import getfullargspec
    from pymorphy2.units.base import BaseAnalyzerUnit
    
    def _get_param_names_311(klass):
        if klass.__init__ is object.__init__:
            return []
        args = getfullargspec(klass.__init__).args
        return sorted(args[1:])
    
    setattr(BaseAnalyzerUnit, '_get_param_names', _get_param_names_311)


pymorphy2_311_hotfix()

In [4]:
articles = pd.read_csv('dataframes/articles.csv')
comments = pd.read_csv('dataframes/comments.csv')

In [5]:
comments['published_datetime'] = pd.to_datetime(comments['published_datetime'])

comments.dtypes

article_id                          int64
author                             object
published_datetime    datetime64[ns, UTC]
text                               object
votes                               int64
dtype: object

In [6]:
comments.sample(10)

,article_id,author,published_datetime,text,votes
3864,778036,RostislavDugin,2023-12-02 12:35:44+00:00,"""но гениальность идеи и гениальность программи...",0
14776,709494,sshemol,2023-01-08 19:44:19+00:00,"То есть только если сам нажмешь кнопку ""скопир...",5
2597,780852,neverblued,2023-12-25 05:35:35+00:00,"Любопытно, если автору так уж не нравятся техн...",0
3957,777704,liontalks,2023-12-02 09:12:51+00:00,"Подход классический, за исключеним того, что м...",0
10848,737694,Superclip,2023-05-30 13:30:05+00:00,"Но, во-первых, так ли часто такая уверенность ...",0
8855,750586,vvbob,2023-07-27 06:07:54+00:00,"Если говорить серьезно, выбор необходимого дл...",6
1454,787606,Finesse,2024-01-21 02:49:31+00:00,"Это какая должна быть память у разработчика, ч...",1
3755,777420,13werwolf13,2023-12-06 17:52:55+00:00,"""Глупый юзер"" - это вовсе не обязательно глупы...",1
16380,701444,sunnybear,2022-11-24 19:54:35+00:00,"Локальный DNS кэш 15 минут? Нет, не слышали (c)",1
2654,782346,ionicman,2023-12-21 21:39:50+00:00,И он абсолютно прав - если проблему можно реши...,0


In [7]:
articles['published_datetime'] = pd.to_datetime(articles['published_datetime'])
articles['tags'] = articles['tags'].astype(str).apply(ast.literal_eval)

articles.dtypes

id                                  int64
author                             object
published_datetime    datetime64[ns, UTC]
title                              object
url                                object
complexity                         object
reading_time                        int64
views                               int64
tags                               object
votes                               int64
bookmarks                           int64
comments                            int64
dtype: object

In [8]:
articles.sample(10)

,id,author,published_datetime,title,url,complexity,reading_time,views,tags,votes,bookmarks,comments
334,764800,dglazkov,2023-10-02 10:54:47+00:00,React Fiber & Concurrency Part 2 (2),https://habr.com/ru/articles/764800/,Простой,8,3600,"[Веб-разработка, JavaScript, ReactJS]",3,30,0
543,740416,Yauheni_Malashuk,2023-06-07 10:38:27+00:00,Atomic Design в веб-дизайне,https://habr.com/ru/articles/740416/,Средний,10,7200,"[Веб-дизайн, Веб-разработка, Дизайн]",3,61,0
778,710202,aio350,2023-01-12 08:01:02+00:00,Краткий обзор Bun — новой среды выполнения Jav...,https://habr.com/ru/companies/timeweb/articles...,NaN,8,12000,"[Блог компании Timeweb Cloud, Веб-разработка, ...",22,46,6
117,787606,hl0rrrka,2024-01-19 12:30:20+00:00,Почему не любят JavaScript?,https://habr.com/ru/articles/787606/,Простой,5,22000,"[Веб-разработка, JavaScript]",16,73,171
514,744964,SSul,2023-06-30 08:13:51+00:00,Приручаем Liquibase. Как скормить базе данных ...,https://habr.com/ru/companies/simbirsoft/artic...,Средний,8,4800,"[Блог компании SimbirSoft, Веб-разработка, Про...",4,22,9
119,787176,Start_X,2024-01-18 08:42:57+00:00,Неочевидные угрозы: как защититься от атак на ...,https://habr.com/ru/companies/StartX/articles/...,NaN,10,3300,"[Блог компании Start X (EX Антифишинг), Информ...",6,36,10
942,688330,MansikM,2022-09-15 06:00:02+00:00,Как мы решаем проблемы со склонением слов для ...,https://habr.com/ru/companies/rshb/articles/68...,NaN,9,6600,"[Блог компании РСХБ.цифра (Россельхозбанк), Ве...",21,59,6
715,718492,gabenD,2023-02-22 09:09:07+00:00,"Зачем кодить, если можно не кодить? Часть 2: G...",https://habr.com/ru/companies/tochka/articles/...,Простой,7,4100,"[Блог компании Точка, Ненормальное программиро...",2,36,0
648,724914,OlegSpectr,2023-03-28 09:23:57+00:00,Внедряем DevSecOps в процесс разработки. Часть...,https://habr.com/ru/companies/spectr/articles/...,Средний,9,5500,"[Блог компании Spectr, Информационная безопасн...",2,50,2
871,692218,Yoskutik,2022-11-09 08:02:55+00:00,MobX с MVVM упрощает жизнь Frontend разработчи...,https://habr.com/ru/articles/692218/,NaN,16,15000,"[Веб-разработка, JavaScript, ReactJS, TypeScript]",8,69,52


In [9]:
print(
    f"Количество статей: {articles.shape[0]}",
    f"Количество комментариев: {comments.shape[0]}",
    '\n'
    f"Количество уникальных авторов постов: {articles['author'].nunique()}",
    f"Количество уникальных комментаторов: {comments['author'].nunique()}",
    '\n'
    f"Период времени сбора информации: {comments['published_datetime'].min().strftime('%Y-%m-%d')} по {comments['published_datetime'].max().strftime('%Y-%m-%d')}",
    sep='\n'
)


Количество статей: 999
Количество комментариев: 18431

Количество уникальных авторов постов: 564
Количество уникальных комментаторов: 5175

Период времени сбора информации: 2022-08-02 по 2024-02-29


In [10]:
def clean(text: str) -> str:
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


def tokenize(text):
    return word_tokenize(text, language="russian")


stop_words = set()
stop_words |= set(stopwords.words('russian'))  # default corpus
stop_words |= set(json.load(open('stopwords-ru.json', encoding='utf-8')))  # additional corpus
stop_words |= set(chr(i) for i in range(ord('а'), ord('а') + 32))  # ru letters
stop_words |= set(chr(i) for i in range(ord('a'), ord('a') + 26))  # en letters


def purge(tokens: list[str]) -> list[str]:
    return [t for t in tokens if t not in stop_words]


def lemmatizer():
    morphy_lemmatizer = pymorphy2.MorphAnalyzer().parse
    nltk_lemmatizer = WordNetLemmatizer().lemmatize
    
    def wrapper(tokens: list[str]) -> list[str]:
        return [morphy_lemmatizer(nltk_lemmatizer(token))[0].normal_form for token in tokens]
    
    return wrapper


In [11]:
comments['tokens'] = comments['text'].astype(str)

tqdm.pandas(desc='lower')
comments['tokens'] = comments['tokens'].progress_apply(str.lower)

tqdm.pandas(desc='clean')
comments['tokens'] = comments['tokens'].progress_apply(clean)

tqdm.pandas(desc='tokenize')
comments['tokens'] = comments['tokens'].progress_apply(tokenize)

tqdm.pandas(desc='lemmatize')
comments['tokens'] = comments['tokens'].progress_apply(lemmatizer())

tqdm.pandas(desc='purge')
comments['tokens'] = comments['tokens'].progress_apply(purge)

lower:   0%|          | 0/18431 [00:00<?, ?it/s]

clean:   0%|          | 0/18431 [00:00<?, ?it/s]

tokenize:   0%|          | 0/18431 [00:00<?, ?it/s]

lemmatize:   0%|          | 0/18431 [00:00<?, ?it/s]

purge:   0%|          | 0/18431 [00:00<?, ?it/s]

In [16]:
comments.to_pickle('pro_comments.pkl')
articles.to_pickle('pro_articles.pkl')